### Librerías utilizadas

In [1]:
import pandas as pd
import numpy as np
import os
import openpyxl
from openpyxl.utils import get_column_letter

### Variables, listas y diccionarios fundamentales en la reestrucutración de la balanza 

In [2]:
rutaGastos = ".\GASTOS 2025.xlsx"
año = "2025"


meses = {'ENE':'ENERO', 'FEB':'FEBRERO', 'MAR':'MARZO', 'ABR':'ABRIL', 'MAY':'MAYO', 'JUN':'JUNIO', 'JUL':'JULIO', 
         'AGO':'AGOSTO', 'SEPT':'SEPTIEMBRE', 'OCT':'OCTUBRE', 'NOV':'NOVIEMBRE', 'DIC':'DICIEMBRE'}
colsRubros = ['PLANTA 1','PLANTA 3','PLANTA 4','MTO TALLER','MTO EDIFICIO','ALMACEN COMPRAS','DISEÑO','CALIDAD',
              'FACT Y EMBARQ','VENTAS','ADMON']
conceptIndepEnOrden = ['GASTOS FINANCIEROS', 'COMISIONES BANCARIAS', 'INTS PAG A INSTITUCIONES FINANC',
                    'INTS PAGADOS A PF Y PM', 'PERDIDA CAMBIARIA', 'GANANCIA CAMBIARIA']
colsGto =  ['GTO1','GTO2','GTO3','GTO4','GTO5','GTO6','GTO7','GTO8','GTO9','GTO10','GTO11']
    
colsCta =  ['CTA1','CTA2','CTA3','CTA4','CTA5','CTA6','CTA7','CTA8','CTA9','CTA10','CTA11']

### Función para ordenar cadenas de texto tipo XXXX-XXXX-XXXX

In [3]:
def clave_ordenacion(x):
        return int(x.split('-')[0])

### Flujo del programa ordenado en pasos secuenciales. Cada paso es crítico en la elaboración del archivo de gastso acumulados, por lo que el mínimo error implica la detención del flujo del programa, de manera que se debe revisar qué pasó manualmente.

In [6]:
# PASO 1: Leer el archivo Excel.
try:
    archivoExcel = pd.ExcelFile(rutaGastos)
    print("EXITO Paso 1.\n")
except Exception as e:
    raise RuntimeError(f"ERROR PASO 1: No se pudo leer el archivo en ruta. {e}")

    
# PASO 2: Establecer dataframe vacio con nombres de columnas.
try:
    nombresColumnas = pd.read_excel(rutaGastos, sheet_name=archivoExcel.sheet_names[0]).columns
    df = pd.DataFrame(columns=nombresColumnas)
    print("EXITO Paso 2.\n")
except Exception as e:
    raise RuntimeError(f"ERROR PASO 2: Error inesperado. {e}")

totalFilasConcatenar = 0


# PASO 3: Dar mes hasta el que se hará la acumulación de gastos.
try:
    while True:
        print(f"Gastos de los meses de este año {año}:")
        for idx, gastoMes in enumerate(archivoExcel.sheet_names, start=1):
            mes = gastoMes.split(' ')[1]
            print(f"{idx}. {mes}")
        
        # El usario da el número de hoja hasta el que desea hacer la acumulación. EL programa checha que la entrada sea tipo
        # int y que sea un número válido
        try:
            numeroHoja = int(input(f"Seleccione el número del mes hasta el que se hará la acumulación de gastos: "))
        except ValueError:
            print('Valor no válido. Por favor ingresa un número entero.\n')
            continue  # Vuelve al inicio del bucle
        
        if numeroHoja in range(1, len(archivoExcel.sheet_names) + 1):
            break
        else:
            print('Valor fuera de rango. Intenta de nuevo.\n')
    
    # Mes explícito correspondiente al número de hoja elegido
    mesAcum = archivoExcel.sheet_names[numeroHoja-1].split(' ')[1]

    print(f"Comenzando acumulación hasta {meses[mesAcum]}...")
    print("EXITO Paso 3.\n")
except Exception as e:
    raise RuntimeError(f"ERROR PASO 3: Error inesperado. {e}")
    

# PASO 4: Unir los dataframes de las hojas en un solo dataframe. Los conceptos a considerar son los que sí tienen cuenta 
# en la columna CUENTA.
for nombreHoja in archivoExcel.sheet_names[:numeroHoja]:
    try:
        # Intentar leer la hoja específica
        dfHoja = pd.read_excel(rutaGastos, sheet_name=nombreHoja)
        
        # Eliminar filas sin valor en la columna "CUENTA"
        dfHoja.dropna(axis="index", how="any", subset=["CUENTA"], inplace=True)
        
        print(f"La hoja {nombreHoja} sumará {dfHoja.shape[0]} filas a df.")

        # Sumar el número de filas
        totalFilasConcatenar += dfHoja.shape[0]

        # Concatenar con el dataframe principal
        try:
            df = pd.concat([df, dfHoja], ignore_index=True)
        except Exception as e:
            raise RuntimeError(f"ERROR PASO 4: Fallo en concatenar la hoja {nombreHoja}: {e}")

    except Exception as e:
        raise RuntimeError(f"ERROR PASO 4: No se pudo leer la hoja {nombreHoja}: {e}")

# Verificación sobre el número de filas del dataframe principal
if totalFilasConcatenar != df.shape[0]:
    raise RuntimeError("ERROR PASO 4: El número de filas de df no coincide con la suma de las filas de los dfHoja.")
else:
    print("EXITO Paso 4.\n")
    

# PASO 5: Obtener lista de los conceptos sin repetición del dataframe principal. Crear dataframe dfF cuya columana CONCEPTO
# está llena por los conceptos sin repetición y cuyas columnas correspondientes a los rubros tienen los acumulados.
# OBS. MODIFICAR ESTE PASO CUANDO SE TENGAN LOS FOLIOS

try:
    # Conceptos sin repetición
    conceptosSinRepeticion = list(set(df.loc[:, "CONCEPTO"].tolist()))
    print(f"Conceptos sin repeticion del dataframe principal: {len(conceptosSinRepeticion)}.")
except Exception as e:
    raise RuntimeError(f"ERROR PASO 5: No se pudo obtener la lista de conceptos sin repetición. {e}")

try:
    # Diccionario para crear el dataframe final
    dicDfFinal = {columna: [np.nan] * len(conceptosSinRepeticion) for columna in df.columns}
except Exception as e:
    raise RuntimeError(f"ERROR PASO 5: No se pudo inicializar el diccionario para el dataframe final. {e}")

for idx, concepto in enumerate(conceptosSinRepeticion, start=0):
    try:
        # Crear dataframe filtrado por concepto
        dfConcepto = df[df['CONCEPTO'] == concepto]

        ## folios = list(set(dfConcepto.loc[:, "FOLIOi"].tolist()))
        cuentas = list(set(dfConcepto.loc[:, "CUENTA"].tolist()))

        # Verificar que los folios son iguales para el concepto actual
        ## if len(folios) != 1:
        ##    raise RuntimeError(f"ERROR PASO 5: Más de un folio en concepto {concepto}: {folios}")

        ## Verificar que las cuentas son iguales para el concepto actual
        if len(cuentas) != 1:
            raise RuntimeError(f"ERROR PASO 5: Hay más de una cuenta en concepto {concepto}: {cuentas}")

        # Agregar el único folio a FOLIOi y FOLIOf, la única cuenta a CUENTA, y el único concepto a 'CONCEPTO'
        ## dicDfFinal['FOLIOi'][idx] = folios[0]
        ## dicDfFinal['FOLIOf'][idx] = folios[0]
        dicDfFinal['CUENTA'][idx] = cuentas[0]
        dicDfFinal['CONCEPTO'][idx] = concepto

        # Suma de los valores de cada columna (rubro)
        for colRubro in colsRubros:
            # El if se ejecuta si existe almenos un elemento de la columna colRubro que no sea nan. El if sirve para no hacer
            # innecesariosm pues si todos los elementos de dicha columna son nan entonces no hay que sumar nada
            if not dfConcepto[colRubro].isna().all():
                dicDfFinal[colRubro][idx] = dfConcepto[colRubro].sum().round(2)    
    except Exception as e:
        raise RuntimeError(f"ERROR PASO 5: Falló el procesamiento del concepto '{concepto}'. {e}")
            
# Crear dataframe final
try:
    dfF = pd.DataFrame(dicDfFinal)
    print("EXITO Paso 5.\n")
except Exception as e:
    raise RuntimeError(f"ERROR PASO 5: No se pudo crear el dataframe final dfF. {e}")
    

# PASO 6: Obtener columna TOTAL GASTOS para los conceptos independientes.
for conceptIndep in conceptIndepEnOrden:
    try:
        # Ver el concepto independiente está en el dataframe final dfF
        aparicionesConcepIndep = len(dfF[dfF["CONCEPTO"] == conceptIndep].index.tolist())
        if aparicionesConcepIndep != 1:
            raise RuntimeError(f"ERROR PASO 6: Concepto {conceptIndep} encontrado 0 veces o en más de una ocasión, total: {aparicionesConcepIndep}")

        dfF.loc[dfF[dfF["CONCEPTO"] == conceptIndep].index.tolist()[0], "TOTAL GASTOS"] = df[df["CONCEPTO"] == conceptIndep]["TOTAL GASTOS"].sum().round(2)
    except Exception as e:
        raise RuntimeError(f"ERROR PASO 6: Fallo el procesamiento del concepto independiente '{conceptIndep}'. {e}")
print("EXITO Paso 6.\n")


# PASO 7: Reordenar dataframe final usando la cuenta (columna CUENTA) de los conceptos.
try:
    # Reordenar filas 
    dfF = dfF.sort_values(by="CUENTA", key=lambda col: col.map(clave_ordenacion))
    dfF.reset_index(drop= True, inplace=True)

    print("EXITO Paso 7.\n")
    
except Exception as e:
    raise RuntimeError(f"ERROR PASO 7: Fallo inesperado: {str(e)}")
    
    
# PASO 8: Eliminar filas con concepto en lista conceptIndepEnOrden. Agregar fila 0, última y las filas de los conceptos
# independientes en el orden que aparecen estos en la lista conceptIndepEnOrden.
# Crear dataframe con conceptos independientes
try:
    dfConcepIndep = dfF[dfF["CONCEPTO"].isin(conceptIndepEnOrden)]
except Exception as e:
    raise RuntimeError(f"ERROR PASO 8: No se pudo filtrar conceptos independientes. {e}")

# Eliminar filas de conceptos independientes del dataframe final
try:
    dfF.drop(dfConcepIndep.index.tolist(), axis=0, inplace=True)
except Exception as e:
    raise RuntimeError(f"ERROR PASO 8: No se pudieron eliminar las filas de conceptos independientes. {e}")

# Agregar fila 0
try:
    dfFila0 = pd.DataFrame(data=[[np.nan]*len(dfF.columns)], columns=dfF.columns)
    dfFila0.loc[0, "CONCEPTO"] = f"ACUMULADO A {meses[mesAcum]}"
except KeyError as e:
    raise RuntimeError(f"ERROR PASO 8: No se pudo asignar el concepto a la fila 0. {e}")
except Exception as e:
    raise RuntimeError(f"ERROR PASO 8: Error al crear la fila 0. {e}")

# Agregar fila 'TOTAL ACUMULADO'
try:
    dfFilaUltima = pd.DataFrame(data=[[np.nan]*len(dfF.columns)], columns=dfF.columns)
    dfFilaUltima.loc[0, "CONCEPTO"] = f"TOTAL ACUMULADO A {mesAcum} {año}"
except KeyError as e:
    raise RuntimeError(f"ERROR PASO 8: No se pudo asignar el concepto a la fila final. {e}")
except Exception as e:
    raise RuntimeError(f"ERROR PASO 8: Error al crear la fila final. {e}")

# Concatenar filas al dataframe final
try:
    dfF = pd.concat([dfFila0, dfF], ignore_index=True)
    dfF = pd.concat([dfF, dfFilaUltima], ignore_index=True)
except Exception as e:
    raise RuntimeError(f"ERROR PASO 8: No se pudieron concatenar las filas al dataframe final. {e}")
    
# Concatenar filas del dataframe dfConcepIndep en el orden que aparecen en la lista conceptIndepEnOrden
for conceptIndep in conceptIndepEnOrden:
    try:
        dfF = pd.concat([dfF, dfConcepIndep[dfConcepIndep["CONCEPTO"] == conceptIndep]], ignore_index=True)
    except Exception as e:
        raise RuntimeError(f"ERROR PASO 8: Fallo en concatenar fila con concepto independiente '{conceptIndep}'. {e}")
        
print("EXITO Paso 8.")



# PASO 9: Cálculos en dataframe.
filaTotalAcum = dfF[dfF["CONCEPTO"] == f"TOTAL ACUMULADO A {mesAcum} {año}"].index.tolist()[0]

# ---- Cálculo totales por fila (suma por fila en TOTAL GASTOS) y por columna (suma en TOTAL ACUM... por rubro)
# Por fila
dfF.loc[1:filaTotalAcum-1, 'TOTAL GASTOS'] = dfF.loc[1:filaTotalAcum-1, colsRubros].sum(axis=1).round(2)

# Por rubro en columna + columna 'TOTAL GASTOS'
for columna in colsRubros:
    dfF.loc[filaTotalAcum, columna] = dfF.loc[1:filaTotalAcum-1, columna].sum().round(2)
    
dfF.loc[filaTotalAcum, 'TOTAL GASTOS'] = dfF.loc[1:filaTotalAcum-1, 'TOTAL GASTOS'].sum().round(2)

# ---- Obtener porcentaje GTO y CTA
# Porcentaje GTO
for k,columna in enumerate(colsGto):
    for i in range(1,filaTotalAcum):
        dfF.loc[i,columna] = ((dfF.loc[i,colsRubros[k]]/dfF.loc[filaTotalAcum, colsRubros[k]])*100).round(2)
        
# Porcentaje CTA
for k,columna in enumerate(colsCta):
    for i in range(1,filaTotalAcum):
        dfF.loc[i,columna] = ((dfF.loc[i,colsRubros[k]]/dfF.loc[i, 'TOTAL GASTOS'])*100).round(2)
        
# ---- Obtener porcentaje columna TOTAL %
for i in range(1, filaTotalAcum):
    dfF.loc[i,'TOTAL %'] = ((dfF.loc[i,'TOTAL GASTOS']/dfF.loc[filaTotalAcum,'TOTAL GASTOS'])*100).round(2)
    
# ---- Obtener suma de los porcentages de cada columna GTO y en la columna TOTAL %
# Columnas GTO
for columna in colsGto:
    dfF.loc[filaTotalAcum, columna] = dfF.loc[1:filaTotalAcum-1, columna].sum().round(1)

#Columna TOTAL %
dfF.loc[filaTotalAcum,'TOTAL %'] = dfF.loc[1:filaTotalAcum-1,'TOTAL %'].sum().round(1)   

# ---- Obtener suma de los porcentajes columna EXTRA
dfF.loc[1:filaTotalAcum-1,'EXTRA'] = dfF.loc[1:filaTotalAcum-1, colsCta].sum(axis=1).round(1)

print("EXITO: Paso 9.\n")



# PASO 10: Guardar datos en archivo Excel y modificar aspecto
# Ruta salida
rutaSalida = fr'.\Gastos acumulado a {mesAcum} 2025 .xlsx'

try:
    # Si el archivo existe, se elimina para evitar problemas con hojas antiguas
    if os.path.exists(rutaSalida):
        os.remove(rutaSalida)

    # Crear ExcelWriter usando openpyxl
    with pd.ExcelWriter(rutaSalida, engine='openpyxl') as writer:
        # Guardar el DataFrame en la hoja 'GTOS ACUM'
        dfF.to_excel(writer, sheet_name=f'GTOS ACUM A {mesAcum}', index=False)

        # Obtener la hoja de trabajo
        worksheet = writer.sheets[f'GTOS ACUM A {mesAcum}']

        # Ajustar el ancho de las columnas
        for i, col in enumerate(dfF.columns):
            try:
                max_len = max(
                    dfF[col].astype(str).map(len).max(),  # Longitud máxima de datos
                    len(str(col))                        # Longitud del nombre de la columna
                )
            except Exception:
                max_len = len(str(col))  # Si hay error, ajustar al nombre de la columna

            col_letter = get_column_letter(i + 1)  # Excel es 1-indexado
            worksheet.column_dimensions[col_letter].width = max_len + 4

    print(f"Archivo '{rutaSalida}' guardado y sobrescrito con ajuste de columnas.")
    print("EXITO Paso 10.")

except Exception as e:
    raise RuntimeError(f"ERROR PASO 10: No se pudo guardar el archivo Excel. {e}")

EXITO Paso 1.

EXITO Paso 2.

Gastos de los meses de este año 2025:
1. ENE
2. FEB
3. MAR
4. ABR
5. MAY
6. JUN
7. JUL
8. AGO
9. SEPT
Seleccione el número del mes hasta el que se hará la acumulación de gastos: 9
Comenzando acumulación hasta SEPTIEMBRE...
EXITO Paso 3.

La hoja GTOS ENE sumará 67 filas a df.
La hoja GTOS FEB sumará 74 filas a df.
La hoja GTOS MAR sumará 78 filas a df.
La hoja GTOS ABR sumará 79 filas a df.
La hoja GTOS MAY sumará 80 filas a df.
La hoja GTOS JUN sumará 83 filas a df.
La hoja GTOS JUL sumará 83 filas a df.
La hoja GTOS AGO sumará 85 filas a df.
La hoja GTOS SEPT sumará 85 filas a df.
EXITO Paso 4.

Conceptos sin repeticion del dataframe principal: 85.
EXITO Paso 5.

EXITO Paso 6.

EXITO Paso 7.

EXITO Paso 8.


C:\Users\mauvp\AppData\Local\Temp\ipykernel_5348\3540833055.py:235: RuntimeWarning: invalid value encountered in scalar divide
  dfF.loc[i,columna] = ((dfF.loc[i,colsRubros[k]]/dfF.loc[i, 'TOTAL GASTOS'])*100).round(2)


EXITO: Paso 9.

Archivo '.\Gastos acumulado a SEPT 2025 .xlsx' guardado y sobrescrito con ajuste de columnas.
EXITO Paso 10.
